<a href="https://colab.research.google.com/github/hamza123232/Machine-Learning/blob/main/t5_2_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.7 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd



data_values = []
data_qas = []
data_qas_answer = []
for n in data_set['context']:
    data_values.append({'context': n})
for i in data_set['qas']:
    for n in i:
        for a in n['answers']:
            data_qas.append(a)
        data_qas_answer.append(n)
data_values_df = pd.DataFrame(data_values)
data_qas_answer_df = pd.DataFrame(data_qas_answer)
data_qas_answer_df.drop('answers', axis=1, inplace=True)
merged_df = data_values_df.join(data_qas_answer_df)
prompts = merged_df.join(pd.DataFrame(data_qas))
prompts = prompts.drop(['id', 'is_impossible', 'answer_start'], axis=1)
prompts['input_text'] = prompts.apply(lambda row: f"{row['context']} {row['question']} {row['text']}", axis=1)

prompts_file = prompts.to_csv("prompts_dataset.txt", header=False, index=False)


In [ ]:
list_a = [1, 2, 3, 4, 5]
list_a.sort(lambda x: x-1)

In [44]:
data_set = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data.json')
for i in data_set['qas']:
    for n in i:
      print(n)
        for a in n['answers']:
          pass

,context,qas
0,In this module MonetX is fetching data of conf...,"[{'id': '00001', 'is_impossible': True, 'quest..."
1,In the monitoring module MonetX is monitoring ...,"[{'id': '00002', 'is_impossible': True, 'quest..."
2,In IPAM module MonetX is scanning through NMAP...,"[{'id': '00004', 'is_impossible': True, 'quest..."
3,In IPAM dashboard there is widget called Ip av...,"[{'id': '00005', 'is_impossible': True, 'quest..."
4,In IPAM dashboard there is widget called subne...,"[{'id': '00006', 'is_impossible': True, 'quest..."
...,...,...
70,In IPAM montX have ability to add DNS server a...,"[{'id': '00104', 'is_impossible': True, 'quest..."
71,Data Center Capacity Management\nIn DCM module...,"[{'id': '00105', 'is_impossible': True, 'quest..."
72,Network monitoring is the use of a system that...,"[{'id': '00106', 'is_impossible': True, 'quest..."
73,in admin page you can see the failed devices a...,"[{'id': '00107', 'is_impossible': True, 'quest..."


In [ ]:
prompts = pd.read_csv("prompts_dataset.txt",names=["context","question", "text",'input_text'])


In [ ]:
prompts

,context,question,text,input_text
0,In this module MonetX is fetching data of conf...,What is use of IPAM in MonetX?,In this module MonetX is fetching data of conf...,In this module MonetX is fetching data of conf...
1,In the monitoring module MonetX is monitoring ...,How are top 10 subnet by IP address are displa...,In the monitoring module MonetX is monitoring ...,In the monitoring module MonetX is monitoring ...
2,In IPAM module MonetX is scanning through NMAP...,What is TCP open port graph displaying?,In IPAM module MonetX is Scanning through NMAP...,In IPAM module MonetX is scanning through NMAP...
3,In IPAM dashboard there is widget called Ip av...,What is Ip availability summary showing?,In IPAM dashboard there is widget called Ip av...,In IPAM dashboard there is widget called Ip av...
4,In IPAM dashboard there is widget called subne...,What is subnet summary showing?,In IPAM dashboard there is widget called subne...,In IPAM dashboard there is widget called subne...
...,...,...,...,...
70,In IPAM montX have ability to add DNS server a...,DNS Services mean?,In IPAM montX have ability to add DNS server a...,In IPAM montX have ability to add DNS server a...
71,Data Center Capacity Management\nIn DCM module...,What is DCM?,Data Center Capacity Management\nIn DCM module...,Data Center Capacity Management\nIn DCM module...
72,Network monitoring is the use of a system that...,What is monitoring?,Network monitoring is the use of a system that...,Network monitoring is the use of a system that...
73,in admin page you can see the failed devices a...,What do failed devices display?,in admin page you can see the failed devices a...,in admin page you can see the failed devices a...


In [ ]:
import re
def clean_text(text):
    text = re.sub(r',', '', text)
    text = re.sub(r'\'', '', text)
    text = re.sub(r'\"', '', text)
    text = re.sub(r'\(', '', text)
    text = re.sub(r'\)', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'“', '', text)
    text = re.sub(r'”', '', text)
    text = re.sub(r'’', '', text)
    text = re.sub(r'\.', '', text)
    text = re.sub(r';', '', text)
    text = re.sub(r':', '', text)
    text = re.sub(r'\-', '', text)
    return text

In [ ]:
prompts['context'] = prompts['context'].apply(clean_text)

prompts['question'] = prompts['question'].apply(clean_text)
prompts['text'] = prompts['text'].apply(clean_text)
prompts['input_text'] = prompts['input_text'].apply(clean_text)

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor

batch_size = 8
num_of_epochs = 10

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)

model.to(device)

Running on the GPU


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
optimizer = Adafactor(model.parameters(), 
                      lr=1e-3, 
                      eps=(1e-30, 1e-3), 
                      clip_threshold=1.0,
                      decay_rate=-0.8, 
                      beta1=None, 
                      weight_decay=0.0, 
                      relative_step=False,
                      scale_parameter=False, 
                      warmup_init=False)

model.train()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
def progress(loss, value, max=100, bar_width=50):
    progress_value = int(value * bar_width / max)
    bar = '[' + '#' * progress_value + ' ' * (bar_width - progress_value) + ']'
    return f"Batch loss: {loss}\n{bar} {value}/{max}"


In [ ]:
loss_per_10_steps = []
num_of_epochs = 20
for epoch in range(1, num_of_epochs + 1):
    print(f"Running epoch: {epoch}")
    running_loss = 0
    num_of_batches = len(prompts) // batch_size
    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        input_batch = []
        label_batch = []
        new_df = prompts[i * batch_size : i * batch_size + batch_size]

        for _, row in new_df.iterrows():
            input_text = "WebNLG: " + row["question"] + "</s>"
            target_text = row["text"] + "</s>"
            input_batch.append(input_text)
            label_batch.append(target_text)

        input_batch = tokenizer.batch_encode_plus(input_batch, padding=True, max_length=400, return_tensors="pt")["input_ids"]
        label_batch = tokenizer.batch_encode_plus(label_batch, padding=True, max_length=400, return_tensors="pt")["input_ids"]
        input_batch = input_batch.to(device)
        label_batch = label_batch.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_batch, labels=label_batch)
        loss = outputs.loss
        loss_num = loss.item()
        logits = outputs.logits
        running_loss += loss_num

        if i % 10 == 0:
            loss_per_10_steps.append(loss_num)

        out.update(progress(loss_num, i, num_of_batches + 1))

        loss.backward()

        optimizer.step()

    running_loss = running_loss / int(num_of_batches)
    print(f"Epoch: {epoch}, Running loss: {running_loss}")

Running epoch: 1


'Batch loss: 2.5341391563415527\n[########################################          ] 8/10'

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch: 1, Running loss: 4.432981093724568
Running epoch: 2


'Batch loss: 1.6549521684646606\n[########################################          ] 8/10'

...

Epoch: 18, Running loss: 0.05549146276381281
Running epoch: 19


'Batch loss: 0.046688567847013474\n[########################################          ] 8/10'

Epoch: 19, Running loss: 0.031984097686492734
Running epoch: 20


'Batch loss: 0.05539368465542793\n[########################################          ] 8/10'

Epoch: 20, Running loss: 0.02438791525653667


In [ ]:
model.save_pretrained('/content/t5_text_model')
tokenizer.save_pretrained('/content/t5_text_tokenizer')

('/content/t5_text_tokenizer/tokenizer_config.json',
 '/content/t5_text_tokenizer/special_tokens_map.json',
 '/content/t5_text_tokenizer/spiece.model',
 '/content/t5_text_tokenizer/added_tokens.json')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_directory = '/content/t5_text_model'
tokenizer_directory = '/content/t5_text_tokenizer'

tokenizer = T5Tokenizer.from_pretrained(tokenizer_directory)
model = T5ForConditionalGeneration.from_pretrained(model_directory, return_dict=True)


In [25]:
!pip install shutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [27]:
import shutil
shutil.make_archive('t55_text_model', 'zip', model_directory)

'/content/t55_text_model.zip'

In [69]:
shutil.make_archive('/content/model_and_tokenizer.h5', 'zip', tokenizer_directory)

'/content/model_and_tokenizer.h5.zip'

In [ ]:
def generate(text,model,tokenizer):
   model.eval()
   input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), 
                               return_tensors="pt")  #"WebNLG:{} </s>".format()
   outputs = model.generate(input_ids)
   return tokenizer.decode(outputs[0])

In [41]:
def generate_text(prompt, model, tokenizer,max_length,temperature):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(input_ids,do_sample=True, max_length=max_length,temperature=temperature)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = generated_text[:max_length]
    return generated_text

generated_text = generate_text("what is montex?", model, tokenizer, max_length=900,temperature=1.0)
print(generated_text)

In this module MonetX is fetching data of configured IP on the network devices physical interfaces SVi and Virtual interfaces LIKE HSRP VRRP & LB virtual IP It is also s the interface description vlan name subnets mask and the status of the physical interface up or down The benefit of this module is to track any ip from the dashboard & does not need to login to device in monetX it have capability to add DHCP server as a node and fetch the information of both ip history with time series titles like Scope name Lease used addresses free addresses etc and you can manually and dynamically discover subnets in your environment to get there IP details like where it is configured Mac address and which ports are open on that IP address and it is also maintainin the ip history with time series if these is MAC change on specific IP it will prompt


In [46]:
from transformers import TFAutoModel
model = TFAutoModel.from_pretrained(model_directory,from_pt=True)
h5_model_path = '/content/t5_text_model/model_t5_text.h5'
model.save_weights(h5_model_path)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5Model: ['lm_head.weight', 'encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5Model for predictions without further training.


In [59]:
from google.colab import files

files.download('/content/model_t5_text.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
import requests
from tqdm import tqdm
import os
def download_with_progress(url, file_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # Adjust the block size as per your preference
    
    progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, ncols=80)
    progress_bar.set_description('Downloading')

    with open(file_path, 'wb') as file:
        for data in response.iter_content(block_size):
            file.write(data)
            progress_bar.update(len(data))
    
    progress_bar.close()

# Usage
file_url = 'http://example.com/model_t5_text.h5'
file_path = '/content/model_t5_text.h5'
download_with_progress(file_url, file_path)



Downloading: : 1.26kB [00:00, 2.84kB/s]


In [54]:
!pip install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
import os
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

def save_to_drive(file_path, drive_folder_id):
    credentials_path = '/content/drive/MyDrive/credentials.json'  # Replace with your own credentials file path
    credentials = None

    if os.path.exists(credentials_path):
        credentials = credentials_path

    service = build('drive', 'v3', credentials=credentials)

    file_metadata = {
        'name': os.path.basename(file_path),
        'parents': [drive_folder_id]
    }
    media = MediaFileUpload(file_path)

    service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()


In [56]:
file_path = '/content/model_t5_text.h5'
drive_folder_id = '1f5kpbT5k8in-Xq9HB_7iox_Nz3yiKOdf'  # Replace with your own folder ID

save_to_drive(file_path, drive_folder_id)

RefreshError: ignored

In [58]:
from google.colab import files

files.download('/content/t55_text_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
import json
import h5py
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer_directory = '/content/t5_text_tokenizer'
model_file = '/content/model_and_tokenizer.h5'

# Create the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_directory)

# Save the model and tokenizer in a single file
model_dict = {"model_state_dict": model.state_dict(), "tokenizer_config": tokenizer.config}
with h5py.File(model_file, 'w') as hf:
    for key, value in model_dict.items():
        if key == 'tokenizer_config':
            hf.attrs[key] = json.dumps(value.to_dict())
        else:
            hf.create_dataset(key, data=value.cpu().numpy())


AttributeError: ignored

In [62]:
import h5py
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_directory = '/content/t5_text_model'
tokenizer_directory = '/content/t5_text_tokenizer'
output_h5_file = '/content/model_and_tokenizer.h5'

tokenizer = T5Tokenizer.from_pretrained(tokenizer_directory)
model = T5ForConditionalGeneration.from_pretrained(model_directory, return_dict=True)

# Create an H5 file
with h5py.File(output_h5_file, 'w') as hf:
    # Save the model weights
    model_state_dict = model.state_dict()
    for key in model_state_dict:
        hf.create_dataset(f'model/{key}', data=model_state_dict[key].numpy())
    
    # Save the tokenizer data
    tokenizer_config = tokenizer.config.to_dict()
    hf.attrs['tokenizer/config'] = str(tokenizer_config)
    tokenizer_files = tokenizer.files
    for file_name in tokenizer_files:
        file_path = f"{tokenizer_directory}/{file_name}"
        with open(file_path, 'rb') as file:
            hf.create_dataset(f'tokenizer/{file_name}', data=file.read())

print(f"Model and tokenizer saved to {output_h5_file}")


AttributeError: ignored

In [67]:
import os
import h5py
import base64
from transformers import T5ForConditionalGeneration, T5TokenizerFast

model_directory = '/content/t5_text_model'
tokenizer_directory = '/content/t5_text_tokenizer'
output_h5_file = '/content/model_and_tokenizer.h5'

tokenizer = T5TokenizerFast.from_pretrained(tokenizer_directory)
model = T5ForConditionalGeneration.from_pretrained(model_directory, return_dict=True)

# Create an H5 file
with h5py.File(output_h5_file, 'w') as hf:
    # Save the model weights
    model_state_dict = model.state_dict()
    for key in model_state_dict:
        hf.create_dataset(f'model/{key}', data=model_state_dict[key].numpy())

    # Save the tokenizer options
    tokenizer_options = tokenizer.init_kwargs['additional_special_tokens']
    hf.attrs['tokenizer/options'] = str(tokenizer_options)

    # Save the tokenizer special tokens
    special_tokens = tokenizer.special_tokens_map
    for token_name, token_value in special_tokens.items():
        hf.attrs[f'tokenizer/special_tokens/{token_name}'] = str(token_value)

    # Save the tokenizer files
    tokenizer_files = os.listdir(tokenizer_directory)
    for file_name in tokenizer_files:
        file_path = f"{tokenizer_directory}/{file_name}"
        with open(file_path, 'rb') as file:
            file_data = file.read()
            encoded_data = base64.b64encode(file_data).decode('utf-8')
            hf.create_dataset(f'tokenizer/{file_name}', data=encoded_data)

print(f"Model and tokenizer saved to {output_h5_file}")


Model and tokenizer saved to /content/model_and_tokenizer.h5


In [68]:
from google.colab import files

files.download('/content/model_and_tokenizer.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
from google.colab import files

files.download('/content/model_and_tokenizer.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
!pip install --upgrade gupload

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.4 require

In [ ]:
from pydrive.auth import GoogleAuth
from google.colab import auth

# Authenticate and create the PyDrive client.
auth.authenticate_user()